In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=fcf184ec290f2b0dd7481f7dbf8c3892c82a4c54aa87906c7a15e516e844897a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
pip install yfinance pandas sqlalchemy

In [9]:
import numpy as np

1.DATA COLLECTION


In [23]:
import yfinance as yf
import pandas as pd


# Define the stock symbol and date range
symbol = 'MSFT'  # Example: STOCK TECHNOLOGY
start_date = '2024-01-01'
end_date = '2024-09-30'

# Download the stock data using yfinance
stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1d')

# Reset index to have 'Date' as a column
stock_data.reset_index(inplace=True)

# Rename columns to match your desired schema
stock_data.rename(columns={
    'Date': 'trade_date',
    'Open': 'open_price',
    'High': 'high_price',
    'Low': 'low_price',
    'Close': 'close_price',
    'Volume': 'volume'
}, inplace=True)



[*********************100%***********************]  1 of 1 completed


In [24]:
print(stock_data.head())

  trade_date  open_price  high_price   low_price  close_price   Adj Close  \
0 2024-01-02  373.859985  375.899994  366.769989   370.869995  368.854095   
1 2024-01-03  369.010010  373.260010  368.510010   370.600006  368.585571   
2 2024-01-04  370.670013  373.100006  367.170013   367.940002  365.940002   
3 2024-01-05  368.970001  372.059998  366.500000   367.750000  365.751038   
4 2024-01-08  369.299988  375.200012  369.010010   374.690002  372.653351   

     volume  
0  25258600  
1  23083500  
2  20901500  
3  20987000  
4  23134000  


2.DATA PREPOCESSING


In [25]:
# Check for missing values
print(stock_data.isnull().sum())

trade_date     0
open_price     0
high_price     0
low_price      0
close_price    0
Adj Close      0
volume         0
dtype: int64


In [26]:
# Fill missing values if necessary, e.g., using forward fill
stock_data.fillna(method='ffill', inplace=True)

<ipython-input-26-cb3fcad91f3d>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='ffill', inplace=True)


In [30]:
# Convert index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Ensure all columns are numeric
stock_data = stock_data.apply(pd.to_numeric, errors='coerce')

In [31]:
## Normalise Data
# Example: Min-Max normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(stock_data), columns=stock_data.columns, index=stock_data.index)



In [32]:

print(scaled_data.head())


                               Open      High       Low     Close    Volume
Datetime                                                                   
2024-09-17 09:30:00-04:00  0.469482  0.526976  0.390454  0.422502  1.000000
2024-09-17 09:31:00-04:00  0.413147  0.449078  0.351410  0.292518  0.201309
2024-09-17 09:32:00-04:00  0.291077  0.087957  0.069416  0.009336  0.316165
2024-09-17 09:33:00-04:00  0.000000  0.041973  0.019522  0.208952  0.223991
2024-09-17 09:34:00-04:00  0.194832  0.000000  0.082431  0.000000  0.157287


3.FEATURE ENGINEERING

In [33]:
# Daily price change
stock_data['Price Change'] = stock_data['Close'].pct_change()


In [35]:
#RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

stock_data['RSI'] = calculate_rsi(stock_data)



In [36]:
#VOLATITY
# Calculate daily returns
stock_data['Daily Return'] = stock_data['Close'].pct_change()

# Calculate rolling volatility (standard deviation)
stock_data['Volatility'] = stock_data['Daily Return'].rolling(window=21).std()



In [37]:
# Calculate moving averages
stock_data['SMA_20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()


In [39]:
## BOLLINGER BANDS
def calculate_bollinger_bands(data, window=20, num_sd=2):
    sma = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Bollinger Upper'] = sma + (rolling_std * num_sd)
    data['Bollinger Lower'] = sma - (rolling_std * num_sd)
    return data

stock_data = calculate_bollinger_bands(stock_data)


In [40]:
print(stock_data.head())


                                 Open        High         Low       Close  \
Datetime                                                                    
2024-09-17 09:30:00-04:00  215.750000  216.110001  215.399994  215.630005   
2024-09-17 09:31:00-04:00  215.630005  215.979904  215.309998  215.350006   
2024-09-17 09:32:00-04:00  215.369995  215.376801  214.660004  214.740005   
2024-09-17 09:33:00-04:00  214.750000  215.300003  214.544998  215.169998   
2024-09-17 09:34:00-04:00  215.164993  215.229904  214.690002  214.719894   

                            Volume  Price Change        RSI  Daily Return  \
Datetime                                                                    
2024-09-17 09:30:00-04:00  1703475           NaN        NaN           NaN   
2024-09-17 09:31:00-04:00   342925     -0.001299   0.000000     -0.001299   
2024-09-17 09:32:00-04:00   538579     -0.002833   0.000000     -0.002833   
2024-09-17 09:33:00-04:00   381563      0.002002  32.575399      0.002002  